In [1]:
import cv2
import os
import matplotlib.pyplot as plt

In [364]:
result_blue = [8,2,3,4,4,7,5,3,5,6]
result_yellow = [5,4,6,2,2,1,3,4,5,2]

In [783]:
list_f = os.listdir('./CoinCounting/')
for ind,value in enumerate(list_f):
    img = cv2.imread('./CoinCounting/'+value)
    size = img.shape[0]*img.shape[1]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    mask_blue = cv2.inRange(img,(40,20,20),(110,255,255))
    mask_yellow = cv2.inRange(img,(-50,100,150),(60,255,255))

    kernel = np.ones((3,3),np.uint8)
    mask_blue = cv2.erode(mask_blue,kernel,iterations = 11)
    mask_yellow = cv2.erode(mask_yellow,kernel,iterations = 1)
    mask_blue = cv2.dilate(mask_blue,kernel,iterations = 4)

    kernel_d = np.zeros((5,5), dtype=np.uint8)
    for i in range(len(kernel_d)):
        for j in range(len(kernel_d)):
            if(i + j == len(kernel_d)-1):
                kernel_d[i][j] = 1

    mask_blue = cv2.erode(mask_blue,kernel_d,iterations = 4)
    mask_blue = cv2.dilate(mask_blue,kernel_d,iterations = 5)


    contours,hierarchy = cv2.findContours(mask_blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours,key=cv2.contourArea,reverse=True)
    # cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
    count_blue = 0
    size_coin = 0
    last_half_coin = [0,0,0,0]
    last_half = False
    img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if(w*h > 0.002*size):
            if(h - 80 < w  < h + 80):
                if(size_coin == 0):
                    cv2.putText(img,str(int(count_blue)),(x,y-5),cv2.FONT_HERSHEY_PLAIN,4, (0,0,255), 3, cv2.LINE_AA)
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 5)
                    count_blue = count_blue + 1
                    size_coin = w*h
                else:
                    if(w*h > 0.4*size_coin):
                        cv2.putText(img,str(int(count_blue)),(x,y-5),cv2.FONT_HERSHEY_PLAIN,4, (0,0,255), 3, cv2.LINE_AA)
                        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 5)
                        count_blue = count_blue + 1
                    else:
                        if(last_half == False):
                            last_half_coin = [x,y,w,h]
                            last_half = True
                        else:
                            if(last_half_coin[0] < x < last_half_coin[0]+last_half_coin[2]):
                                if(last_half_coin[0] < x ):
                                    cv2.putText(img,str(int(count_blue)),(last_half_coin[0],  last_half_coin[1]-5),cv2.FONT_HERSHEY_PLAIN,4, (0,0,255), 3, cv2.LINE_AA) 
                                    cv2.rectangle(img, (last_half_coin[0],  last_half_coin[1]), ( x + w,  y + h), (0, 0, 255), 5)
                                else:
                                    cv2.putText(img,str(int(count_blue)),(x,y-5),cv2.FONT_HERSHEY_PLAIN,4, (0,0,255), 3, cv2.LINE_AA)
                                    cv2.rectangle(img, (x,  y), (last_half_coin[0] + last_half_coin[2] , last_half_coin[1] + last_half_coin[3]), (0, 0, 255), 5)
                                count_blue = count_blue + 1
                                last_half = False
                            else:
                                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 5)
                                count_blue = count_blue + 1
    if(last_half):
        cv2.putText(img,str(int(count_blue)),(last_half_coin[0],  last_half_coin[1]-5),cv2.FONT_HERSHEY_PLAIN,4, (0,0,255), 3, cv2.LINE_AA)
        cv2.rectangle(img, (last_half_coin[0],  last_half_coin[1]), (last_half_coin[0] + last_half_coin[2] , last_half_coin[1] + last_half_coin[3]), (0, 0, 255), 5)
        count_blue = count_blue + 1

    contours,hierarchy = cv2.findContours(mask_yellow, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours,key=cv2.contourArea,reverse=True)
    count_yellow = 0
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if(w*h > 0.002*size):
            if(h - 80 < w  < h + 80):
                cv2.putText(img,str(int(count_yellow)),(x,y-5),cv2.FONT_HERSHEY_PLAIN,4, (255,0,0), 3, cv2.LINE_AA)
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 5)
                count_yellow = count_yellow + 1

    img = cv2.resize(img, (600,600), interpolation = cv2.INTER_AREA)
    mask_blue = cv2.resize(mask_blue, (600,600), interpolation = cv2.INTER_AREA)
    mask_yellow = cv2.resize(mask_yellow, (600,600), interpolation = cv2.INTER_AREA)


    print(value[:-4])
    print('Blue coin : ',end='')
    print(result_blue[ind],end='')
    print(' --------> ',end='')
    print(count_blue)
    print('Yellow coin : ',end='')
    print(result_yellow[ind],end='')
    print(' --------> ',end='')
    print(count_yellow)


coin1
Blue coin : 8 --------> 8
Yellow coin : 5 --------> 5
coin10
Blue coin : 2 --------> 2
Yellow coin : 4 --------> 4
coin2
Blue coin : 3 --------> 3
Yellow coin : 6 --------> 6
coin3
Blue coin : 4 --------> 4
Yellow coin : 2 --------> 2
coin4
Blue coin : 4 --------> 4
Yellow coin : 2 --------> 2
coin5
Blue coin : 7 --------> 7
Yellow coin : 1 --------> 1
coin6
Blue coin : 5 --------> 5
Yellow coin : 3 --------> 3
coin7
Blue coin : 3 --------> 3
Yellow coin : 4 --------> 4
coin8
Blue coin : 5 --------> 5
Yellow coin : 5 --------> 5
coin9
Blue coin : 6 --------> 6
Yellow coin : 2 --------> 2


In [804]:
img = cv2.imread('./CoinCounting/coin9.jpg')
size = img.shape[0]*img.shape[1]
img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

mask_blue = cv2.inRange(img,(40,20,20),(110,255,255))
mask_yellow = cv2.inRange(img,(-50,100,150),(60,255,255))

kernel = np.ones((3,3),np.uint8)
mask_blue = cv2.erode(mask_blue,kernel,iterations = 11)
mask_yellow = cv2.erode(mask_yellow,kernel,iterations = 1)
mask_blue = cv2.dilate(mask_blue,kernel,iterations = 4)

kernel_d = np.zeros((5,5), dtype=np.uint8)
for i in range(len(kernel_d)):
    for j in range(len(kernel_d)):
        if(i + j == len(kernel_d)-1):
            kernel_d[i][j] = 1

mask_blue = cv2.erode(mask_blue,kernel_d,iterations = 4)
mask_blue = cv2.dilate(mask_blue,kernel_d,iterations = 5)


contours,hierarchy = cv2.findContours(mask_blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
contours = sorted(contours,key=cv2.contourArea,reverse=True)
# cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
count_blue = 0
size_coin = 0
last_half_coin = [0,0,0,0]
last_half = False
img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if(w*h > 0.002*size):
        if(h - 80 < w  < h + 80):
            if(size_coin == 0):
                cv2.putText(img,str(int(count_blue)),(x,y-5),cv2.FONT_HERSHEY_PLAIN,4, (0,0,255), 3, cv2.LINE_AA)
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 5)
                count_blue = count_blue + 1
                size_coin = w*h
            else:
                if(w*h > 0.4*size_coin):
                    cv2.putText(img,str(int(count_blue)),(x,y-5),cv2.FONT_HERSHEY_PLAIN,4, (0,0,255), 3, cv2.LINE_AA)
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 5)
                    count_blue = count_blue + 1
                else:
                    if(last_half == False):
                        last_half_coin = [x,y,w,h]
                        last_half = True
                    else:
                        if(last_half_coin[0] < x < last_half_coin[0]+last_half_coin[2]):
                            if(last_half_coin[0] < x ):
                                cv2.putText(img,str(int(count_blue)),(last_half_coin[0],  last_half_coin[1]-5),cv2.FONT_HERSHEY_PLAIN,4, (0,0,255), 3, cv2.LINE_AA) 
                                cv2.rectangle(img, (last_half_coin[0],  last_half_coin[1]), ( x + w,  y + h), (0, 0, 255), 5)
                            else:
                                cv2.putText(img,str(int(count_blue)),(x,y-5),cv2.FONT_HERSHEY_PLAIN,4, (0,0,255), 3, cv2.LINE_AA)
                                cv2.rectangle(img, (x,  y), (last_half_coin[0] + last_half_coin[2] , last_half_coin[1] + last_half_coin[3]), (0, 0, 255), 5)
                            count_blue = count_blue + 1
                            last_half = False
                        else:
                            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 5)
                            count_blue = count_blue + 1
if(last_half):
    cv2.putText(img,str(int(count_blue)),(last_half_coin[0],  last_half_coin[1]-5),cv2.FONT_HERSHEY_PLAIN,4, (0,0,255), 3, cv2.LINE_AA)
    cv2.rectangle(img, (last_half_coin[0],  last_half_coin[1]), (last_half_coin[0] + last_half_coin[2] , last_half_coin[1] + last_half_coin[3]), (0, 0, 255), 5)
    count_blue = count_blue + 1

contours,hierarchy = cv2.findContours(mask_yellow, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
contours = sorted(contours,key=cv2.contourArea,reverse=True)
count_yellow = 0
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if(w*h > 0.002*size):
        if(h - 80 < w  < h + 80):
            cv2.putText(img,str(int(count_yellow)),(x,y-5),cv2.FONT_HERSHEY_PLAIN,4, (255,0,0), 3, cv2.LINE_AA)
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 5)
            count_yellow = count_yellow + 1

img = cv2.resize(img, (600,600), interpolation = cv2.INTER_AREA)
mask_blue = cv2.resize(mask_blue, (600,600), interpolation = cv2.INTER_AREA)
mask_yellow = cv2.resize(mask_yellow, (600,600), interpolation = cv2.INTER_AREA)


cv2.imshow('mask_blue',mask_blue)
cv2.moveWindow('mask_blue',0,10)
cv2.imshow('mask_yellow',mask_yellow)
cv2.moveWindow('mask_yellow', 950, 10)
cv2.imshow('img',img)
cv2.moveWindow('img', 450, 200)
cv2.waitKey(0)
cv2.destroyAllWindows()